In [1]:
import numpy as np
import pandas as pd
from sklearn import (
    feature_extraction,
    linear_model,
    model_selection,
    preprocessing,
    cluster,
    multiclass,
    svm,
)

In [2]:
pip install jupyter-black jupyter

Note: you may need to restart the kernel to use updated packages.


In [3]:
# for black formating
import jupyter_black  # pip install jupyter-black jupyter

jupyter_black.load()
# ??jupyter_black.load #show config

In [4]:
df = pd.read_csv("/kaggle/input/p5-nlp/stackoverflowCleaned.csv", index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118442 entries, 0 to 118441
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Id           118442 non-null  int64 
 1   Title        118442 non-null  object
 2   Body         118442 non-null  object
 3   0            118440 non-null  object
 4   1            118383 non-null  object
 5   2            118292 non-null  object
 6   3            118292 non-null  object
 7   4            76389 non-null   object
 8   5            4 non-null       object
 9   Title_clean  118389 non-null  object
 10  Code         88202 non-null   object
 11  Body_clean   118392 non-null  object
dtypes: int64(1), object(11)
memory usage: 11.7+ MB


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
df["0"]

0                java
1                 git
2                 git
3                 git
4                 c++
             ...     
118437         iphone
118438    google-maps
118439     javascript
118440           java
118441             c#
Name: 0, Length: 118442, dtype: object

In [6]:
from ipywidgets import interact


@interact(i=(0, df.shape[0] - 1))
def test_tr_print(i):
    print(
        f'Title:{df["Title"][i]}\n{"*"*30}\nTitle_clean:{df["Title_clean"][i]}\n{"*"*30}\nBody:{df["Body"][i]}\n{"*"*30}\nBody_clean:{df["Body_clean"][i]}\n{"*"*30}\nCode:{df["Code"][i]}\n{"*"*30}\nTags:{df.loc[i,"0"]} {df.loc[i,"1"]} {df.loc[i,"2"]} {df.loc[i,"3"]} {df.loc[i,"4"]}'
    )

interactive(children=(IntSlider(value=59220, description='i', max=118441), Output()), _dom_classes=('widget-in…

In [7]:
sum(df["Title_clean"].isna())

53

In [8]:
df[df["Title_clean"].isna()][
    ["Title", "Title_clean", "Body", "Body_clean", "0", "1", "2", "3", "4"]
]

,Title,Title_clean,Body,Body_clean,0,1,2,3,4
302,Is < faster than <=?,NaN,<p>Is <code>if (a &lt; 901)</code> faster than...,example performance change loop code suppose g...,c++,c,performance,assembly,relational-operators
393,What is setup.py?,NaN,<p>What is <code>setup.py</code> and how can i...,configure,python,pypi,setup.py,python-packaging,NaN
1437,What is related_name used for?,NaN,<p>What is the <code>related_name</code> argum...,argument manytomanyfield foreignkey field exam...,python,django,foreign-keys,many-to-many,NaN
4062,"What is ""android:allowBackup""?",NaN,"<p>Since the <a href=""http://tools.android.com...",adt preview version version lint warning tell ...,android,adt,compiler-warnings,android-lint,android-backup-service
4870,Why is 0 < -0x80000000?,NaN,<p>I have below a simple program:</p>\n\n<pre>...,program condition if(bal int32_min work change...,c,signed,numeric-limits,numeric-conversion,NaN
5007,Make UINavigationBar transparent,NaN,<p>How do you make a <strong>UINavigationBar t...,uinavigationbar want bar item remain,ios,iphone,objective-c,uinavigationbar,transparency
7467,"vim ""modifiable"" is off",NaN,<p>I am trying to create a new file with NERDT...,try create file nerdtree hit key create file m...,vim,vi,macvim,nerdtree,NaN
9122,~x + ~y == ~(x + y) is always false?,NaN,<p>Does this code always evaluate to false? Bo...,code evaluate variable complement sign int fee...,c,bit-manipulation,signed,twos-complement,NaN
12942,'do...while' vs. 'while',NaN,<blockquote>\n <p><strong>Possible Duplicates...,duplicate vs. use loop program year work year ...,c#,c++,c,while-loop,do-while
15258,"Hashable, immutable",NaN,"<p>From a recent SO question (see <a href=""htt...",question create dictionary python index list r...,python,data-structures,hash,immutability,NaN


In [9]:
import math

math.pow(np.nan, 0)

1.0

In [10]:
print(sum(df["Body_clean"].isna()))
df[df["Body_clean"].isna()][["Title", "Title_clean", "Body", "Body_clean", "Code"]]

50


,Title,Title_clean,Body,Body_clean,Code
5,What and where are the stack and heap?,stack heap,<ul>\n<li>What are the stack and heap?</li>\n<...,NaN,NaN
168,What is The Rule of Three?,rule,<ul>\n<li>What does <em>copying an object</em>...,NaN,NaN
828,Why are these constructs using pre and post-in...,construct pre behavior,<pre><code>#include &lt;stdio.h&gt;\n\nint mai...,NaN,#include <stdio.h>\n\nint main(void)\n{\n in...
1207,How to use ADB Shell when Multiple Devices are...,use adb shell multiple device connect fail err...,<pre><code>$ adb --help\n</code></pre>\n<hr />...,NaN,$ adb --help\n -s SERIAL use device with give...
1640,What are the advantages of list initialization...,advantage list initialization brace,<pre><code>MyClass a1 {a}; // clearer and ...,NaN,MyClass a1 {a}; // clearer and less error-...
2129,How to use GROUP BY to concatenate strings in ...,use group concatenate string sql server,<p>How do I get:</p>\n\n<pre><code>id Na...,NaN,id Name Value\n1 A ...
2242,curl: (60) SSL certificate problem: unable to ...,curl ssl certificate problem certificate,<pre><code>root@sclrdev:/home/sclr/certs/Fresh...,NaN,root@sclrdev:/home/sclr/certs/FreshCerts# curl...
3014,Picasso v/s Imageloader v/s Fresco vs Glide vs...,picasso v s imageloader v s fresco vs glide vs...,<h2>Findings:</h2>\n<ol>\n<li>Difference betwe...,NaN,NaN
5990,Algorithm to implement a word cloud like Wordle,algorithm implement word cloud like wordle,<h2>Context</h2>\n\n<ul>\n<li>Take a look at W...,NaN,NaN
8810,Checking if array is multidimensional or not?,check array,<ol>\n<li>What is the most efficient way to ch...,NaN,is_array()


In [11]:
df.shape[0]

118442

In [12]:
LONG = False
for min_df in [0.01, 0.05, 0.005, 0.0005, 0.00001] if LONG else [0.0005]:
    for max_df in [0.95, 0.995, 0.9995] if LONG else [0.9995]:
        print(f"\n{'*'*30}\n for min_df:{min_df}; max_df:{max_df}")
        count_vectorizer = feature_extraction.text.CountVectorizer(
            min_df=min_df, max_df=max_df, max_features=None
        )

        title_vectors = count_vectorizer.fit_transform(df["Title_clean"].fillna(""))
        body_vectors = count_vectorizer.fit_transform(df["Body_clean"].fillna(""))
        print(
            f"\ttokens unique:\n\t\t{title_vectors.shape[1]} dans title\n\t\t{body_vectors.shape[1]} dans body"
        )


******************************
 for min_df:0.0005; max_df:0.9995
	tokens unique:
		1330 dans title
		3553 dans body


In [13]:
title_vectors.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [14]:
title_tfid_transformer = feature_extraction.text.TfidfTransformer(
    norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False
).fit(title_vectors)
title_tfid = title_tfid_transformer.transform(title_vectors)
title_tfid.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
body_tfid = feature_extraction.text.TfidfTransformer(
    norm="l2", use_idf=True, smooth_idf=True, sublinear_tf=False
).fit_transform(body_vectors)
body_tfid.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
def tag_is_in(df: pd.DataFrame(), tag: str, nb_cols: int = 6):
    return eval("|".join(f'(df["{i}"] == "{tag}")' for i in range(0, nb_cols)))


tag_is_in(df, "javascript")

0         False
1         False
2         False
3         False
4         False
          ...  
118437    False
118438    False
118439     True
118440    False
118441    False
Length: 118442, dtype: bool

In [17]:
body_tfid[1000:1001]

<1x3553 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [18]:
OvsRClf_js = multiclass.OneVsRestClassifier(svm.SVC(random_state=42)).fit(
    body_tfid[0:5000], tag_is_in(df[0:5000], "javascript")
)

In [19]:
pred = OvsRClf_js.predict(body_tfid[5000:6050])
pred

array([False, False, False, ..., False, False, False])

In [20]:
tag_is_in(df[5000:6050], "javascript")

5000    False
5001    False
5002    False
5003    False
5004    False
        ...  
6045    False
6046    False
6047    False
6048    False
6049    False
Length: 1050, dtype: bool

In [21]:
# def evaluate_pred(reels=tag_is_in(df[5000:6050], "javascript"), preds=pred):
def evaluate_pred(reels, preds):
    fauxVrai = 0
    vraiFaux = 0
    bonVrai = 0
    bonFaux = 0
    bon = 0
    faux = 0
    for reel, pred in zip(reels, preds):
        # print(a, b)
        # print("test", a == b)
        if reel and pred:
            bonVrai += 1
        if reel and (not pred):
            fauxVrai += 1
        if (not reel) and pred:
            vraiFaux += 1
        if (not reel) and (not pred):
            bonFaux += 1
        if reel == pred:
            bon += 1
        else:
            faux += 1
    print(f"taille {len(preds)}")
    print(
        f"predit faux alors que vrai:{fauxVrai}\npredit vrai alors que faux:{vraiFaux}\nbonVrai:{bonVrai}(/{sum(preds)}) bonFaux:{bonFaux}(/{len(preds) - sum(preds)})"
    )
    print(f"bon:{bon} faux:{faux}")
    # print(f"accuracy:{(bonVrai+bonFaux)/(fauxVrai+vraiFaux):.2%}")
    print(f"Accuracy:{(bon)/(len(preds)):.2%}")
    # print(f"Recall: {bonVrai/(bonVrai+vraiFaux):.2%}")
    print(
        f"Recall: {bonFaux/(len(preds) - sum(preds)):.2%}"
    )  # -> 100% Recall: No false negatives, every negative prediction is correct.
    print(
        f"Precision: {bonVrai/sum(preds):.2%}"
    )  # -> 100% Precision: No false positives, every positive prediction is correct
    # print(f"Precision2: {bonVrai/(bonVrai+fauxVrai):.2%}")


evaluate_pred(reels=tag_is_in(df[5000:6050], "javascript"), preds=pred)

taille 1050
predit faux alors que vrai:77
predit vrai alors que faux:13
bonVrai:44(/57) bonFaux:916(/993)
bon:960 faux:90
Accuracy:91.43%
Recall: 92.25%
Precision: 77.19%


In [22]:
#OvsRClf_title_js = multiclass.OneVsRestClassifier(svm.SVC(random_state=42)).fit(
#    title_tfid[0:1000], tag_is_in(df[0:1000], "javascript")
#)

In [23]:
#pred_title_js = OvsRClf_title_js.predict(title_tfid[1000:1050])

In [24]:
#evaluate_pred(reels=tag_is_in(df[1000:1050], "javascript"), preds=pred_title_js)

In [25]:
def OvR_train(
    start: int = 0, end: int = 50000, tag: str = "java", to_pred=(50001, 100000)
):
    reels = tag_is_in(df[start:end], tag)
    OvsRClf_body = multiclass.OneVsRestClassifier(svm.SVC(random_state=42)).fit(
        body_tfid[start:end], reels
    )
    OvsRClf_title = multiclass.OneVsRestClassifier(svm.SVC(random_state=42)).fit(
        title_tfid[start:end], reels
    )
    return {
        "pred_body": OvsRClf_body.predict(body_tfid[to_pred[0] : to_pred[1]]),
        "pred_title": OvsRClf_title.predict(title_tfid[to_pred[0] : to_pred[1]]),
        "reels_p": tag_is_in(df[to_pred[0] : to_pred[1]], tag),
    }


pred_java = OvR_train(start=0, end=10000,tag="java", to_pred=(10001,100000))

In [26]:
evaluate_pred(reels=tag_is_in(df[10001:100000], "java"), preds=pred_java["pred_body"])

taille 89999
predit faux alors que vrai:6958
predit vrai alors que faux:883
bonVrai:3896(/4779) bonFaux:78262(/85220)
bon:82158 faux:7841
Accuracy:91.29%
Recall: 91.84%
Precision: 81.52%


In [27]:
evaluate_pred(reels=tag_is_in(df[10001:100000], "java"), preds=pred_java["pred_title"])

taille 89999
predit faux alors que vrai:6274
predit vrai alors que faux:763
bonVrai:4580(/5343) bonFaux:78382(/84656)
bon:82962 faux:7037
Accuracy:92.18%
Recall: 92.59%
Precision: 85.72%
